# * ETL-Actual MKS : Broadband
    (P, G, H, HH, Z) level

## Parameter

In [2]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

### Check Source Summary

In [4]:
# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
# print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()


try:
    src_cur.execute("""
        SELECT TM_KEY_MTH
            , SUM(SUBS) TOTAL
            , SUM(CASE WHEN ISP = 'TOL' THEN SUBS END) TOL
            , SUM(CASE WHEN ISP = '3BB' THEN SUBS END) "3BB"
            , SUM(CASE WHEN ISP = 'AIS' THEN SUBS END) AIS
            , SUM(CASE WHEN ISP IN ('CAT', 'TOT') THEN SUBS END) NT
            --, SUM(CASE WHEN ISP = 'CAT' THEN SUBS END) CAT
            --, SUM(CASE WHEN ISP = 'TOT' THEN SUBS END) TOT
        FROM CORPNSBOX.FCT_BB_SHARE_SUBS_CCAATT NOLOCK
        WHERE TM_KEY_MTH = (SELECT MAX(TM_KEY_MTH) FROM CORPNSBOX.FCT_BB_SHARE_SUBS_CCAATT NOLOCK)
        GROUP BY TM_KEY_MTH
        ORDER BY 1
    """)
    rows = src_cur.fetchall()
    # print(f'\nCurrent Source Summary...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    mod_col_list = chk_src_df.iloc[:, 1:].columns.tolist()
    for col in mod_col_list:
        chk_src_df[col] = chk_src_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{chk_src_df}')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    # print(f'\n{TDMDBPR_db} : Disconnected')


   TM_KEY_MTH      TOTAL        TOL        3BB        AIS         NT
0      202405  8,440,027  3,082,690  2,167,565  1,765,884  1,423,888


### Check Fact Summary

In [5]:
# Connect : AKPIPRD
tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
tgt_conn = oracledb.connect(tgt_dsn)
# print(f'\n{AKPIPRD_db} : Connected')
tgt_cur = tgt_conn.cursor()


try:
    tgt_cur.execute("""
        SELECT TM_KEY_MTH
            , SUM(CASE WHEN METRIC_CD IN ('VIN00025', 'VIN00026', 'VIN00027', 'VIN00028') THEN METRIC_VALUE END) TOTAL
            , SUM(CASE WHEN METRIC_CD = 'VIN00025' THEN METRIC_VALUE END) TOL
            , SUM(CASE WHEN METRIC_CD = 'VIN00026' THEN METRIC_VALUE END) "3BB"
            , SUM(CASE WHEN METRIC_CD = 'VIN00027' THEN METRIC_VALUE END) AIS
            , SUM(CASE WHEN METRIC_CD = 'VIN00028' THEN METRIC_VALUE END) NT
            --, MAX(LOAD_DATE) LOAD_DATE
        FROM AUTOKPI.FCT_BROADBAND_MKS NOLOCK
        WHERE TM_KEY_MTH = (SELECT MAX(TM_KEY_MTH) FROM AUTOKPI.FCT_BROADBAND_MKS NOLOCK WHERE REMARK IS NULL)
        AND TM_KEY_DAY LIKE '%01'
        AND AREA_TYPE = 'P'
        AND REMARK IS NULL
        GROUP BY TM_KEY_MTH
        ORDER BY 1
    """)
    rows = tgt_cur.fetchall()
    # print(f'\nCurrent Fact Summary...')
    chk_tgt_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in tgt_cur.description])
    mod_col_list = chk_tgt_df.iloc[:, 1:6].columns.tolist()
    for col in mod_col_list:
        chk_tgt_df[col] = chk_tgt_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{chk_tgt_df}')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    tgt_conn.close()
    # print(f'\n{AKPIPRD_db} : Disconnected')


   TM_KEY_MTH      TOTAL        TOL        3BB        AIS         NT
0      202405  8,439,795  3,082,458  2,167,565  1,765,884  1,423,888


### Input Parameter

In [115]:
''' Input Parameter '''

# Create Param
v_param = dict(mth_start = 202405, mth_end = 202405)
v_update_flag = 'Y'
v_target_schema = 'AUTOKPI'
v_target_table = 'FCT_BROADBAND_MKS'

# Create Query
query_delete = f"""
    DELETE {v_target_schema}.{v_target_table} 
    WHERE TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']}
"""

print(f"\nParameter input...\n\n   -> mth_start: {v_param['mth_start']}\n   -> mth_end: {v_param['mth_end']}")
print(f'\nquery_delete...\n{query_delete}')


Parameter input...

   -> mth_start: 202405
   -> mth_end: 202405

query_delete...

    DELETE AUTOKPI.FCT_BROADBAND_MKS 
    WHERE TM_KEY_MTH BETWEEN 202405 AND 202405



## ETL Process...

### Query from DB source to DB Target
    Delete -> Insert

    Source : CORPNSBOX.FCT_BB_SHARE_SUBS_CCAATT
             CDSAPPO.DIM_MOOC_AREA
             CDSAPPO.DIM_TIME
    
    Target : AUTOKPI.FCT_BROADBAND_MKS

In [116]:
job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')


# Read : SQL file
with open('SQL/Import-FCT_BROADBAND_MKS.sql', 'r') as sql_file:
    queries = sql_file.read().split(';')
    query = queries[0].strip()
    sql_file.close()


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()


# Connect : AKPIPRD
tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
tgt_conn = oracledb.connect(tgt_dsn)
print(f'\n{AKPIPRD_db} : Connected')
tgt_cur = tgt_conn.cursor()


try:
    print(f'\nProcessing...')
    
# Create Dataframe
    src_cur.execute(query, v_param)
    rows = src_cur.fetchall()
    print(f'\nCreate Dataframe...')
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> src_df : {src_df.shape[0]} rows, {src_df.shape[1]} columns') 

# Truncate
    # tgt_cur.execute(f'TRUNCATE TABLE {v_target_schema}.{v_target_table}')
    # print(f'\n   -> TRUNCATE : "{v_target_table}" : Done !')

# Delete
    tgt_cur.execute(query_delete)
    print(f'\n   -> DELETE : "{v_target_table}" : Done !')
    
# Insert
    tgt_cur.executemany(f"""
        INSERT INTO {v_target_schema}.{v_target_table}
        (TM_KEY_YR, TM_KEY_MTH, TRUE_TM_KEY_WK, TM_KEY_DAY, METRIC_CD, METRIC_NAME, COMP_CD, VERSION, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, METRIC_VALUE, AGG_TYPE, FREQUENCY, REMARK) 
        VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16)
        """, rows)
    print(f'\n   -> INSERT : "{v_target_table}" : Done !')

    tgt_cur.close()
    tgt_conn.commit()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')

    tgt_conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    
    print(f'\nJob Done !!!')



Job Start... 2024-06-23, 00:15:52

TDMDBPR : Connected

AKPIPRD : Connected

Processing...

Create Dataframe...

   -> src_df : 107970 rows, 16 columns

   -> DELETE : "FCT_BROADBAND_MKS" : Done !

   -> INSERT : "FCT_BROADBAND_MKS" : Done !

TDMDBPR : Disconnected

AKPIPRD : Disconnected

Job Done !!!


In [120]:
''' Example DataFrame '''

# tmp_df = src_df.groupby(['VERSION', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'METRIC_VALUE': 'mean', 'TM_KEY_MTH': 'nunique', 'AREA_TYPE': 'nunique', 'AREA_CD': 'nunique'}).reset_index()
# tmp_df = src_df.groupby(['VERSION', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'TM_KEY_MTH': ['min','max'], 'METRIC_VALUE': 'mean', 'AREA_TYPE': 'nunique', 'AREA_CD': 'nunique'}).reset_index()
tmp_df = src_df.groupby(['VERSION', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'TM_KEY_DAY': ['min','max'], 'METRIC_VALUE': 'mean', 'AREA_TYPE': 'nunique', 'AREA_CD': 'nunique'}).reset_index()
tmp_df

VERSION COMP_CD METRIC_CD                              METRIC_NAME  \
                                                                       
0       A    TRUE  VIN00019         Broadband Subs Share : AIS & 3BB   
1       A    TRUE  VIN00020               Broadband Subs Share : TOL   
2       A    TRUE  VIN00021               Broadband Subs Share : 3BB   
3       A    TRUE  VIN00022               Broadband Subs Share : AIS   
4       A    TRUE  VIN00023                Broadband Subs Share : NT   
5       A    TRUE  VIN00024  Broadband Subs Share (Subs) : AIS & 3BB   
6       A    TRUE  VIN00025        Broadband Subs Share (Subs) : TOL   
7       A    TRUE  VIN00026        Broadband Subs Share (Subs) : 3BB   
8       A    TRUE  VIN00027        Broadband Subs Share (Subs) : AIS   
9       A    TRUE  VIN00028         Broadband Subs Share (Subs) : NT   

  TM_KEY_DAY             METRIC_VALUE AREA_TYPE AREA_CD  
         min       max           mean   nunique nunique  
0   20240501  20240630      47.125324         5     177  
1   20240501  20240630      34.925863         5     177  
2   20240501  20240630      25.847456         5     177  
3   20240501  20240630      21.277867         5     177  
4   20240501  20240630      17.948813         5     177  
5   20240501  20240630  119459.479712         5     177  
6   20240501  20240630   96373.067797         5     177  
7   20240501  20240630   65109.475063         5     177  
8   20240501  20240630   54350.004649         5     177  
9   20240501  20240630   43200.381647         5     177

In [65]:
# 1 + 8 + 65 + 96
# 5487 * 10 * 12